In [1]:
from src import scrape_atjam2025
from src import data_for_spread

In [2]:
data = scrape_atjam2025.main("https://up-t.jp/collabo/atjam2025",0,"atjam2025_data.json")

Scraped and saved at 2025-08-06 19:09:14


In [3]:
data

{'broad_rank_texts': [{'datetime': '2025年08月06日 17:46',
   'name': '愛崎ユウナ',
   'count': 1},
  {'datetime': '2025年08月06日 15:47', 'name': '新庄愛', 'count': 1},
  {'datetime': '2025年08月06日 15:42', 'name': '新庄愛', 'count': 1},
  {'datetime': '2025年08月06日 15:27', 'name': '姫奈', 'count': 1},
  {'datetime': '2025年08月06日 15:27', 'name': '姫奈', 'count': 1},
  {'datetime': '2025年08月06日 13:38', 'name': '新庄愛', 'count': 1},
  {'datetime': '2025年08月06日 12:32', 'name': '姫奈', 'count': 1},
  {'datetime': '2025年08月06日 12:32', 'name': '松尾優花', 'count': 1},
  {'datetime': '2025年08月06日 12:20', 'name': '桐嶋奏歌', 'count': 1},
  {'datetime': '2025年08月06日 10:43', 'name': '紬希わかな', 'count': 1}],
 'rank_table': [{'name': '青木 眞歩', 'point': 455595},
  {'name': 'みおん', 'point': 345138},
  {'name': '足森いづみ', 'point': 311116},
  {'name': '新庄愛', 'point': 281698},
  {'name': '松尾優花', 'point': 259441},
  {'name': 'マクマキ', 'point': 259365},
  {'name': 'YUYU', 'point': 218854},
  {'name': '茅野風花', 'point': 211591},
  {'name': '蜂光うに', '

In [4]:
from bs4 import BeautifulSoup
import requests
html_path="https://up-t.jp/collabo/atjam2025"
resp = requests.get(html_path)
resp.raise_for_status()
html = resp.text
soup = BeautifulSoup(html, "html.parser")

In [5]:
broad_rank_texts = scrape_atjam2025.extract_broad_rank_texts(soup)

In [6]:
broad_rank_texts

[{'datetime': '2025年08月07日 00:29', 'name': 'マクマキ', 'count': 1},
 {'datetime': '2025年08月07日 00:29', 'name': 'マクマキ', 'count': 2},
 {'datetime': '2025年08月07日 00:29', 'name': 'マクマキ', 'count': 1},
 {'datetime': '2025年08月07日 00:29', 'name': 'マクマキ', 'count': 1},
 {'datetime': '2025年08月07日 00:29', 'name': 'マクマキ', 'count': 1},
 {'datetime': '2025年08月07日 00:29', 'name': 'マクマキ', 'count': 1},
 {'datetime': '2025年08月07日 00:21', 'name': '蜂光うに', 'count': 1},
 {'datetime': '2025年08月06日 23:15', 'name': '佐藤 愛唯', 'count': 1},
 {'datetime': '2025年08月06日 23:09', 'name': '新庄愛', 'count': 1},
 {'datetime': '2025年08月06日 23:09', 'name': '新庄愛', 'count': 5}]

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# --- 設定 ---
SERVICE_ACCOUNT_FILE = 'research-181705-deac2d5759c1.json'  # ダウンロードしたサービスアカウント JSON
SPREADSHEET_ID      = '1-xZeooGI0_s7PCmIL_7LNBMeTaZ9_PzoX9lyOOF1WX0'    # スプレッドシート ID
# ----------------

def auth_gspread():
    scope = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
    creds  = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, scope)
    return gspread.authorize(creds)

gc = auth_gspread()


def get_worksheet(name: str):
    """名前でワークシートを開く"""
    sh = gc.open_by_key(SPREADSHEET_ID)
    return sh.worksheet(name)

In [13]:
records = broad_rank_texts
"""
「直近の購入実績」シートへ追加。
    - 過去に同じ(datetime, name, count)が登録済みならスキップ
    - 同一更新内の重複は許容
    - 追加後に「購入日時」で降順ソート
"""
ws = get_worksheet("直近の購入実績")
all_rows = ws.get_all_values()[1:]  # ヘッダ行を除くすべての既存行
existing = {(r[0], r[1], r[2]) for r in all_rows}

to_append = []
for rec in records:
    key = (rec["datetime"], rec["name"], rec["count"])
    if key not in existing:
        to_append.append([rec["datetime"], rec["name"], rec["count"], "test"])

In [14]:
ws.get_all_values()[1:]

[['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 0:30:03'],
 ['2025年08月07日 00:29', 'マクマキ', '2', '2025-08-07 0:30:03'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 0:30:03'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 0:30:03'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 0:30:03'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 0:30:03'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 1:03:01'],
 ['2025年08月07日 00:29', 'マクマキ', '2', '2025-08-07 1:03:01'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 1:03:01'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 1:03:01'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 1:03:01'],
 ['2025年08月07日 00:29', 'マクマキ', '1', '2025-08-07 1:03:01'],
 ['2025年08月07日 00:21', '蜂光うに', '1', '2025-08-07 0:22:01'],
 ['2025年08月07日 00:21', '蜂光うに', '1', '2025-08-07 1:03:01'],
 ['2025年08月06日 23:15', '佐藤 愛唯', '1', '2025-08-06 23:15:02'],
 ['2025年08月06日 23:15', '佐藤 愛唯', '1', '2025-08-07 1:03:01'],
 ['2025年08月06日 23:09', '新庄愛', '1', '2025-08-06 23:10:

In [10]:
existing

{('2025年08月06日 10:43', '紬希わかな', '1'),
 ('2025年08月06日 12:20', '桐嶋奏歌', '1'),
 ('2025年08月06日 12:32', '姫奈', '1'),
 ('2025年08月06日 12:32', '松尾優花', '1'),
 ('2025年08月06日 13:38', '新庄愛', '1'),
 ('2025年08月06日 15:27', '姫奈', '1'),
 ('2025年08月06日 15:42', '新庄愛', '1'),
 ('2025年08月06日 15:47', '新庄愛', '1'),
 ('2025年08月06日 17:46', '愛崎ユウナ', '1'),
 ('2025年08月06日 21:19', '新庄愛', '1'),
 ('2025年08月06日 21:49', '青木 眞歩', '1'),
 ('2025年08月06日 22:53', '松尾優花', '1'),
 ('2025年08月06日 23:09', '新庄愛', '1'),
 ('2025年08月06日 23:09', '新庄愛', '5'),
 ('2025年08月06日 23:09', '新庄愛', '6'),
 ('2025年08月06日 23:15', '佐藤 愛唯', '1'),
 ('2025年08月07日 0:21', '蜂光うに', '1'),
 ('2025年08月07日 0:29', 'マクマキ', '1'),
 ('2025年08月07日 0:29', 'マクマキ', '2')}

In [15]:
for rec in records:
    key = (rec["datetime"], rec["name"], rec["count"])
    if key not in existing:
        print(key)

('2025年08月07日 00:29', 'マクマキ', 1)
('2025年08月07日 00:29', 'マクマキ', 2)
('2025年08月07日 00:29', 'マクマキ', 1)
('2025年08月07日 00:29', 'マクマキ', 1)
('2025年08月07日 00:29', 'マクマキ', 1)
('2025年08月07日 00:29', 'マクマキ', 1)
('2025年08月07日 00:21', '蜂光うに', 1)
('2025年08月06日 23:15', '佐藤 愛唯', 1)
('2025年08月06日 23:09', '新庄愛', 1)
('2025年08月06日 23:09', '新庄愛', 5)


In [9]:
to_append

[['2025年08月07日 00:29', 'マクマキ', 1, 'test'],
 ['2025年08月07日 00:29', 'マクマキ', 2, 'test'],
 ['2025年08月07日 00:29', 'マクマキ', 1, 'test'],
 ['2025年08月07日 00:29', 'マクマキ', 1, 'test'],
 ['2025年08月07日 00:29', 'マクマキ', 1, 'test'],
 ['2025年08月07日 00:29', 'マクマキ', 1, 'test'],
 ['2025年08月07日 00:21', '蜂光うに', 1, 'test'],
 ['2025年08月06日 23:15', '佐藤 愛唯', 1, 'test'],
 ['2025年08月06日 23:09', '新庄愛', 1, 'test'],
 ['2025年08月06日 23:09', '新庄愛', 5, 'test']]

In [ ]:
if to_append:
    ws.append_rows(to_append, value_input_option='USER_ENTERED')

# 購入日時（1列目）で降順ソート
# sort((col_index, 'desc'))
ws.sort((1, 'des'))